# 🔗 Function 4: Join Station Data

## Building the `join_station_data` Function

**Learning Objectives:**
- Understand data joining and merging operations in pandas
- Learn different types of joins (inner, left, right, outer)
- Master the `.merge()` function and join keys
- Handle missing data and validate join results
- Combine datasets from different sources

**Professional Context:**
Data joining is essential for:
- **Combining datasets** - Merge station metadata with sensor readings
- **Data enrichment** - Add geographic coordinates, station names, and location details
- **Analysis preparation** - Create complete datasets with all necessary information
- **Quality assurance** - Identify stations with missing metadata or readings

## Part 1: Understanding Data Joins

### 1.1 What is a Data Join?

**Data joining** is combining two datasets based on a common column (called a "key").

**Example scenario:**
- Dataset 1: **Sensor readings** (station_id, temperature, humidity)
- Dataset 2: **Station metadata** (station_id, station_name, latitude, longitude)
- **Goal**: Combine them to get readings WITH station details

The **join key** is `station_id` - the column that exists in both datasets.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create sample datasets for demonstration
readings_data = pd.DataFrame({
    'station_id': ['STN_001', 'STN_002', 'STN_001', 'STN_003', 'STN_002'],
    'temperature_c': [22.5, 19.3, 23.1, 16.8, 20.1],
    'humidity_percent': [65.2, 78.1, 63.9, 82.3, 76.5],
    'reading_time': ['2024-01-01 08:00', '2024-01-01 08:00', '2024-01-01 12:00', '2024-01-01 08:00', '2024-01-01 12:00']
})

stations_metadata = pd.DataFrame({
    'station_id': ['STN_001', 'STN_002', 'STN_003', 'STN_004'],
    'station_name': ['Downtown Plaza', 'Coastal Park', 'Mountain Ridge', 'Airport Terminal'],
    'latitude': [40.123, 40.789, 41.234, 40.567],
    'longitude': [-74.456, -73.987, -74.567, -74.123],
    'elevation_m': [10, 5, 450, 25]
})

print("READINGS DATA:")
print(readings_data)
print("\nSTATIONS METADATA:")
print(stations_metadata)

print("\nKEY OBSERVATIONS:")
print(f"Stations in readings: {sorted(readings_data['station_id'].unique())}")
print(f"Stations in metadata: {sorted(stations_metadata['station_id'].unique())}")
print(f"STN_004 has metadata but no readings (common scenario)")

### 1.2 Types of Joins

There are four main types of joins:

In [ ]:
print("=== DIFFERENT JOIN TYPES ===")

# 1. INNER JOIN - Only records that exist in BOTH datasets
inner_join = pd.merge(readings_data, stations_metadata, on='station_id', how='inner')
print(f"\n1. INNER JOIN: {len(inner_join)} rows")
print("   Includes only stations with BOTH readings AND metadata")
print(inner_join[['station_id', 'temperature_c', 'station_name']].head())

# 2. LEFT JOIN - All readings, add metadata where available
left_join = pd.merge(readings_data, stations_metadata, on='station_id', how='left')
print(f"\n2. LEFT JOIN: {len(left_join)} rows")
print("   Includes ALL readings, adds metadata where available")
print(left_join[['station_id', 'temperature_c', 'station_name']].head())

# 3. RIGHT JOIN - All stations, add readings where available
right_join = pd.merge(readings_data, stations_metadata, on='station_id', how='right')
print(f"\n3. RIGHT JOIN: {len(right_join)} rows")
print("   Includes ALL stations, adds readings where available")
print(right_join[['station_id', 'temperature_c', 'station_name']].head())

# 4. OUTER JOIN - Everything from both datasets
outer_join = pd.merge(readings_data, stations_metadata, on='station_id', how='outer')
print(f"\n4. OUTER JOIN: {len(outer_join)} rows")
print("   Includes ALL readings AND all stations")
print(outer_join[['station_id', 'temperature_c', 'station_name']].head())

## Part 2: Implementing Data Joins

### 2.1 Basic Merge Operations

The pandas `.merge()` function is the primary tool for joining data:

In [ ]:
# Demonstrate basic merge syntax
print("=== PANDAS MERGE FUNCTION ===")
print("Basic syntax: pd.merge(left_df, right_df, on='key_column', how='join_type')")

# Perform left join (most common for this use case)
joined_data = pd.merge(
    left=readings_data,
    right=stations_metadata, 
    on='station_id',
    how='left'
)

print(f"\nJoined data shape: {joined_data.shape}")
print(f"Columns: {list(joined_data.columns)}")

print("\nJoined data sample:")
display_cols = ['station_id', 'temperature_c', 'station_name', 'latitude', 'longitude']
print(joined_data[display_cols].head())

# Check for missing metadata
missing_metadata = joined_data['station_name'].isna().sum()
print(f"\nReadings with missing metadata: {missing_metadata}")

if missing_metadata > 0:
    missing_stations = joined_data[joined_data['station_name'].isna()]['station_id'].unique()
    print(f"Stations missing metadata: {list(missing_stations)}")

### 2.2 Join Validation and Quality Checking

After joining, always validate the results:

In [ ]:
# Analyze join results
print("=== JOIN VALIDATION ===")

print(f"Original readings: {len(readings_data)}")
print(f"After join: {len(joined_data)}")
print(f"Records with complete data: {joined_data['station_name'].notna().sum()}")

# Data completeness by station
completeness = joined_data.groupby('station_id').agg({
    'temperature_c': 'count',
    'station_name': lambda x: x.notna().all()
}).round(1)
completeness.columns = ['reading_count', 'has_metadata']

print("\nCompleteness by station:")
print(completeness)

# Summary report
complete_stations = completeness[completeness['has_metadata'] == True]
incomplete_stations = completeness[completeness['has_metadata'] == False]

print(f"\nStations with complete metadata: {len(complete_stations)}")
print(f"Stations missing metadata: {len(incomplete_stations)}")

if len(incomplete_stations) > 0:
    affected_readings = incomplete_stations['reading_count'].sum()
    total_readings = completeness['reading_count'].sum()
    print(f"Readings affected: {affected_readings} out of {total_readings} ({100*affected_readings/total_readings:.1f}%)")

## Part 3: Building the Complete Function

### 3.1 Complete Function Implementation

Now let's build the complete function:

In [ ]:
def join_station_data_example(readings_df, stations_df):
    """Example implementation of the join_station_data function."""
    
    print("=" * 50)
    print("JOINING STATION DATA")
    print("=" * 50)
    
    # Input validation
    if readings_df is None or len(readings_df) == 0:
        print("Error: Readings DataFrame is empty or None")
        return pd.DataFrame()
    
    if stations_df is None or len(stations_df) == 0:
        print("Error: Stations DataFrame is empty or None") 
        return pd.DataFrame()
    
    # Check for required join key
    if 'station_id' not in readings_df.columns:
        print("Error: 'station_id' column missing from readings data")
        return pd.DataFrame()
    
    if 'station_id' not in stations_df.columns:
        print("Error: 'station_id' column missing from stations data")
        return pd.DataFrame()
    
    # Print input summary
    print(f"Input data:")
    print(f"  Readings: {len(readings_df)} rows, {len(readings_df.columns)} columns")
    print(f"  Stations: {len(stations_df)} rows, {len(stations_df.columns)} columns")
    
    # Analyze join keys
    readings_stations = set(readings_df['station_id'].unique())
    metadata_stations = set(stations_df['station_id'].unique())
    
    print(f"\nJoin analysis:")
    print(f"  Stations in readings: {len(readings_stations)}")
    print(f"  Stations in metadata: {len(metadata_stations)}")
    print(f"  Stations in both: {len(readings_stations & metadata_stations)}")
    
    if readings_stations - metadata_stations:
        print(f"  ⚠️ Readings without metadata: {readings_stations - metadata_stations}")
    if metadata_stations - readings_stations:
        print(f"  ℹ️ Metadata without readings: {metadata_stations - readings_stations}")
    
    # Perform left join to keep all readings
    print(f"\nPerforming LEFT JOIN (keeping all readings)...")
    result = pd.merge(readings_df, stations_df, on='station_id', how='left')
    
    # Validate results
    missing_metadata_count = result.isnull().any(axis=1).sum()
    complete_records = len(result) - missing_metadata_count
    
    print(f"\nJoin results:")
    print(f"  Total records: {len(result)}")
    print(f"  Complete records: {complete_records}")
    print(f"  Records with missing metadata: {missing_metadata_count}")
    print(f"  Data completeness: {100*complete_records/len(result):.1f}%")
    
    print("\n✅ Station data join completed successfully!")
    
    return result

# Test the function
test_result = join_station_data_example(readings_data, stations_metadata)
print("\n=== FINAL RESULT ===")
print(test_result.head())

## Part 4: Your Implementation Task

### 4.1 Implementation Guidelines

Now implement this function in `src/pandas_basics.py`. Key steps:

```python
def join_station_data(readings_df, stations_df):
    # TODO: Print header
    # TODO: Validate both input DataFrames (check if None or empty)
    # TODO: Check for 'station_id' column in both DataFrames
    # TODO: Print input data summary
    # TODO: Analyze join keys (stations in each dataset)
    # TODO: Perform LEFT JOIN using pd.merge()
    # TODO: Validate join results
    # TODO: Print completion message and statistics
    # TODO: Return joined DataFrame
```

### 4.2 Testing Your Implementation

```bash
uv run pytest tests/test_pandas_basics.py::test_join_station_data -v
```

### 4.3 Key Requirements

- Use **LEFT JOIN** to keep all readings
- Join on `'station_id'` column
- Handle missing metadata gracefully
- Return the complete joined DataFrame
- Provide informative error messages

## 🎯 Summary and Next Steps

### What You've Learned
- How to join datasets using pandas `.merge()`
- Different types of joins and when to use them
- Validating join results and handling missing data
- Professional data integration workflows

### Your Implementation Checklist
- [ ] Validate both input DataFrames
- [ ] Check for required 'station_id' column
- [ ] Analyze join keys before merging
- [ ] Use LEFT JOIN with `how='left'`
- [ ] Report join statistics and data completeness
- [ ] Return properly merged DataFrame

### Next Function
Once implemented and tested, move on to:
**[`05_function_save_processed_data.ipynb`](05_function_save_processed_data.ipynb)**

Where you'll learn to save your processed data for future use!

---

**Remember**: Data joining is like connecting puzzle pieces - make sure the pieces fit together properly! 🧩